## Random Forests  

### Problem motivation

**Recap:** the performance of a particular decision tree  $ {\cal T} $ depends on its structure -- branches from the root to the leaves -- and its content -- boolean functions / dimensions $ \lbrace \left( f_{\boldsymbol{\eta}}(\cdot), d_{\boldsymbol{\eta}} \right) \rbrace $ assigned to the decision nodes $ \lbrace \boldsymbol{\eta} \rbrace $ and e.g. categorical distributions $ \lbrace P_{\boldsymbol{\ell}}(\cdot) \rbrace $ assigned to the prediction nodes $ \lbrace \boldsymbol{\ell} \rbrace $. On the other hand, the structure and content of tree $ {\cal T} $ are tightly coupled with the particular set $ {\cal D} $ of training examples employed to build it. Therefore, the performance of two decision trees $ {\cal T}^{(1)} $ and $ {\cal T}^{(2)} $ built to perform the same classification task, but using slightly different datasets $ {\cal D}^{(1)} $ and $ {\cal D}^{(2)} $, might significantly differ. **Conclusion:** despite being quite easy for a human to interpret how a decision tree performs the classification task, a decision tree alone is not such a reliable classifier.

### Solution approach

We can improve the performance of isolated decision trees by using a randomized ensemble of decision trees. The randomized ensemble is created by means of a technique called *bagging* which includes both:
* **Bootstrapping:** use several randomized versions or bootstraps $ \lbrace {\cal D}^{(b)} \rbrace $, $ b \in \lbrace 1, \ldots, B \rbrace $, of the training dataset $ {\cal D} $ to build different decision trees $ \lbrace {\cal T}^{(b)} \rbrace $ using the CART algorithm {cite}`breiman1984classification`. We can also randomize the CART algorithm parameters. For example, we can select a particular set of features $ {\cal F}^{(b)} \subset \lbrace 1, \ldots, D \rbrace $ to build each tree $ {\cal T}^{(b)} $; and
* **Aggregating:** for a particular observed sample $ \check{\bf x} $, we combine the estimates $ \lbrace \hat{y}^{(b)} \rbrace $ provided by the ensemble $ {\cal E} = \lbrace {\cal T}^{(b)} \rbrace $, $ b \in \lbrace 1, \ldots, B \rbrace $, to produce a final estimate $ \hat{y} $.

That leads to one of the most powerful existing classifiers today to work with *unstructured* data: the random forests (RFs). 

### Bias/variance tradeoff

Remember that, by assumption, the training dataset -- and also the testing dataset -- of the type $$ {\cal D} = \bigcup_{i=1}^{N} \lbrace \left( {\bf x}_{i}, y_{i} \right) \rbrace, $$ in which $ {\bf x}_{i} \in {\cal X} $ and $ y_{i} \in {\cal Y} $, contains several ($ N \gg 0 $) realizations of the random vectors / variables $ {\bf X}, Y $ jointly distributed according to some unknown distribution $ p^{\ast}({\bf x}, y) $. Note that the prediction $ \hat{y} $ provided by a trained classifier $ h:{\cal X} \rightarrow {\cal Y} $ for an arbitrary feature vector $ \check{\bf x} $ -- i.e. a particular sample of the random vector $ {\bf X} $ -- is a realization of the random variable $ \hat{Y} = h({\bf X}) $. Hence, the classifier prediction itself can be modeled as a random variable. Now, let the random variable
\begin{eqnarray}
Z &\triangleq& \left[ Y = h({\bf X}) \right] \nonumber \\
&\equiv& \left[ Y = \hat{Y} \right] \nonumber \\
\end{eqnarray}
indicate a classification match which is distributed according to the p.m.f $ P_{Z}(z) = Pr \lbrace Z=z \rbrace $, $ z \in \lbrace 0, 1 \rbrace $, such that $ Pr \lbrace Z = 0 \rbrace \equiv Pr \lbrace Y \neq h({\bf X}) \rbrace $ and $ Pr \lbrace Z = 1 \rbrace \equiv Pr \lbrace Y = h({\bf X}) \rbrace $ correspond respectively to the true classification error $ CE_{true} $  and to the true classification accuracy $ ACC_{true} $ of the classifier $ h(\cdot) $. Thus, a particular classification match $ z_{i} = \left[ y_{i} = \hat{y}_{i} \right] $ with $ \hat{y}_{i} = h({\bf x}_{i}) $ committed by the classifier $ h (\cdot) $ for the $i$-th data item $ {\bf x}_{i} $ in $ {\cal D} $ is a realization of the random variable $ Z $. We can therefore rewrite the training classification error $ CE_{train} $ and accuracy $ ACC_{train} $ as
\begin{eqnarray}
 ACC_{train} &=& 1 - CE_{train} \nonumber \\
 &=& \frac{1}{N} \sum_{i=1}^{N} z_{i} \nonumber
\end{eqnarray}
such that $ CE_{train} \rightarrow CE_{true} $ and $ ACC_{train} \rightarrow ACC_{true} $ when the training dataset $ {\cal D} $ grows unbounded, i.e. $ N \rightarrow \infty $. Since $ CE_{train} $ and $ ACC_{train} $ are empirical approximations to the true classification error $ CE_{true} $ and classification accuracy $ ACC_{true} $, respectively, relying on the samples $ \lbrace z_{i} \rbrace $, $ i \in \lbrace 1, \ldots, N \rbrace $, they are also realizations of random variables which are in turn functions of the random variable $ Z $. A similar analysis can be done for the testing classification error $ CE_{test} $ and accuracy $ ACC_{test} $.

To summarize, the empirical training -- or testing -- performance metrics of a classifier $ h(\cdot) $ are realizations of random variables which depend on the particular samples of the training -- or testing -- datasets. Putting more straightforwardly, different datasets lead to different empirical performance metrics $ CE_{train} $, $ ACC_{train} $, $ CE_{test} $ and $ ACC_{test} $.

Now, let us consider the impact of the model complexity on the classification performance. The model complexity is a measure of the flexibility of a model. For example, it can correspond to the number of nearest neighbors $ K $ employed by a KNN classifier or to the depth of the binary tree employed by a DT classifier.

Let $ h_{{\cal D}} (\cdot) $ represent a classifier trained with a particular training dataset ${\cal D}$. Furthermore, let us define the mean squared error (MSE) for a given feature vector $ {\bf x} $ as
$$
MSE({\bf x}) = E \lbrace \left( Y - h_{{\cal D}}({\bf X}) \right)^{2} \mid {\bf X}={\bf x}\rbrace
$$
with the expected value taken over all possible (random) datasets $ {\cal D} $ containing training examples $ \left( {\bf x}_{i}, y_{i} \right) $ drawn from a distribution $ p^{\ast} $. We offer without proof that the MSE can be decomposed into three components as follows
$$
MSE({\bf x}) = \underbrace{\left( E \lbrace h_{{\cal D}}({\bf x}) - \bar{h}({\bf x}) \rbrace \right)^{2}}_{\mbox{Bias}^{2}} + \underbrace{E \lbrace \left( h_{{\cal D}}({\bf x}) - \bar{h}({\bf x}) \right)^{2} \rbrace}_{\mbox{Variance}} + \mbox{Irreducible term},
$$
where the  $\bar{h}({\bf x}) \triangleq E \lbrace h_{{\cal D}}({\bf X}) \rbrace $ is the expected classifier computed over all possible datasets $ {\cal D} $. 

In general, the model complexity[^footnote3] affects these components as follows
* The bias component captures the inherent error of the classifier (even if trainned with an infinite size dataset $ {\cal D}^{\infty} $) and it always decreases $\downarrow$ as the model complexity increases $\uparrow$.
* The variance component captures how much the classifier changes if trainned with different datasets and it always increases $\uparrow$ as the model complexity increases $\uparrow$.
* The irreducible component does not depend on the model complexity and it is related to the data itself.

[^footnote3]: A complex model has more flexibility -- e.g. more parameters or bigger structure -- to adapt itself to the training data than a simpler one.

Thus, as shown in {numref}`bias_variance_tradeoff1a_fig`, the bias and variance components contribute differently to the total prediction error. In particular, there is an optimal complexity that minimizes the prediction error. The bias-variance tradeoff consists exactly in choosing the optimal model complexity taking into account the bias and the variance contributions to the total prediction error.

```{figure} /images/classification/bias_variance_tradeoff.svg
---
height: 320px
name: bias_variance_tradeoff1a_fig
align: left
---
Bias and variance contributing to the total error.
```

{numref}`bias_variance_tradeoff1b_fig` illustrates how the training and testing prediction errors behave as the model complexity increases. As indicated in the figure, as the model complexity increases, the bias component always decreases and the variance component always increases when composing the testing prediction error. Specifically, as the model complexity starts increasing, we observe a decrease in the testing prediction error as the squared bias component quickly decreases. The testing prediction error continues to decrease as the model complexity gets higher, but at some point it starts increasing as the variance component gets too high due to model overfitting, i.e. the model starts overfitting the training data. In this case, despite fitting better and better to the training data, the model starts loosing its ability to properly generalize and deal with *unseen* samples from a testing dataset. Note though that the training prediction error always decreases as the model complexity increases. 

```{figure} /images/classification/e2.png
---
height: 320px
name: bias_variance_tradeoff1b_fig
align: left
---
The prediction error as a function of model complexity (borrowed from {cite}`hastie2009elements`). The performance curves in light blue and light red are associated with different training datasets. More specifically, each light blue curve was created by training classifiers with increasing model complexities using the same training dataset and by evaluating then their prediction errors. On the other hand, the light red curves were created by evaluating the prediction error of the trained classifiers using a testing dataset. The bold curves in blue and red represent respectively the average of the training and testing performance curves. Lastly, bold curves indicate the prediction error bias whereas light tone curves indicate its variance, i.e. how spread the performance curves associated with different training datasets are around the corresponding average curve.
```

````{margin}
```{note}
The results shown in {numref}`bias_variance_tradeoff1b_fig` suggest that the optimal model complexity corresponds to the point -- or interval -- in which the bias stops decreasing. Before reaching this point the model is too simple and underfits the training dataset and after this point the model is too flexible, overfits the training dataset and, as a consequence, loses its ability to properly generalize across the testing dataset.
```

```{note}
{numref}`bias_variance_tradeoff1b_fig` also indicates that, for a fixed model complexity, the performance can be significantly different for two distinct training datasets. However, in average, the classifiers obtained by different training datasets can perform consistently good (or poor).
```
````

### Bootstrapping

Based on the previous discussion regarding the bias-variance trade off, we can improve the performance of an isolated DT classifier by exploring the average behavior -- bias of the prediction error -- of several DT classifiers trained using different datasets. First, let us create $ B $ randomized versions -- bootstraps -- $ \lbrace {\cal D}^{(b)} \rbrace $, $ b \in \lbrace 1, 2, \ldots, B \rbrace $, of the training dataset
\begin{eqnarray}
{\cal D} &=& \bigcup_{i=1}^{N} \lbrace \underbrace{\left( {\bf x}_{i}, y_{i} \right)}_{{\cal D}_{i}} \rbrace \nonumber \\
&=& \lbrace {\cal D}_{1}, {\cal D}_{2}, {\cal D}_{3}, {\cal D}_{4}, {\cal D}_{5}, {\cal D}_{6}, {\cal D}_{7}, \ldots , {\cal D}_{N} \rbrace \nonumber
\end{eqnarray}
by sampling pairs $ {\cal D}_{i} \triangleq \left( {\bf x}_{i}, y_{i} \right) $ from $ {\cal D} $ with replacement -- which means that multiple repetitions of the same pair are allowed in each bootstrap. Specifically, let the p.m.f. $ U_{\cal A}(a) $ denote an uniform distribution over the elements of a finite alphabet $ {\cal A} $ such that $ U_{\cal A}(a) = \frac{1}{|{\cal A}|} $, $ \forall a \in {\cal A} $. We build the $b$-th bootstrap by making $$ {\cal D}^{(b)}_{j} \gets {\cal D}_{i_{j}} $$ where the index $ i_{j} $ is drawn from an uniform distribution over the set $ \lbrace 1, \ldots, N \rbrace $, i.e. $$ i_{j} \sim U_{\lbrace 1, \ldots, N \rbrace}(\xi), $$ for each $ j \in \lbrace 1, 2, \ldots N \rbrace $. 

We repeat this procedure for $ b \in \lbrace 1, \ldots, B \rbrace $ to obtain $ B $ bootstraps as illustrated below
\begin{eqnarray}
{\cal D}^{(1)} &=& \underbrace{\lbrace {\cal D}_{9}, {\cal D}_{8}, {\cal D}_{9}, {\cal D}_{6}, {\cal D}_{2}, {\cal D}_{7}, {\cal D}_{1}, \ldots \rbrace}_{N} \nonumber \\
{\cal D}^{(2)} &=& \underbrace{\lbrace {\cal D}_{11}, {\cal D}_{6}, {\cal D}_{7}, {\cal D}_{7}, {\cal D}_{1}, {\cal D}_{14}, {\cal D}_{20}, \ldots \rbrace}_{N} \nonumber \\
{\cal D}^{(3)} &=& \underbrace{\lbrace {\cal D}_{3}, {\cal D}_{9}, {\cal D}_{2}, {\cal D}_{4}, {\cal D}_{17}, {\cal D}_{11}, {\cal D}_{16}, \ldots \rbrace}_{N} \nonumber \\
&\vdots& \nonumber \\
{\cal D}^{(B)} &=& \underbrace{\lbrace {\cal D}_{5}, {\cal D}_{18}, {\cal D}_{7}, {\cal D}_{5}, {\cal D}_{1}, {\cal D}_{12}, {\cal D}_{7}, \ldots \rbrace}_{N}. \nonumber
\end{eqnarray}

```{prf:remark}
Note that each pair $ {\cal D}_{i} \in {\cal D} $, $ \forall i \in \lbrace 1, \ldots, N \rbrace $, can appear multiple times in the same bootstrap. For $ B \rightarrow \infty $, only $ 1 - \frac{1}{e} \approx 63.21 $ \% of the original samples will be presented at each bootstrap. Note also that bootstraps follow aleatory sequences with respect to the sequence of pairs $ \lbrace {\cal D}_{1}, {\cal D}_{2}, {\cal D}_{3}, \ldots \rbrace $ in the original dataset $ {\cal D} $.
```

We can follow a similar procedure to create a subset of features $ {\cal F}^{(b)} \subset \lbrace 1, 2, \ldots, D \rbrace $ but restricted to a smaller number of features such that $ | {\cal F}^{(b)} | < D $.

Finally, we create an ensemble $ {\cal E} = \lbrace {\cal T}^{(b)} \rbrace $, $ b \in \lbrace 1, 2, \ldots, B \rbrace $, of decision trees using the CART algorithm. In particular, the $b$-th decision tree is created by running the CART algorithm $$ {\cal T}^{(b)} \gets CART({\cal D}^{(b)}, \infty; k_{max}; \delta_{min}; {\cal F}^{(b)}) $$ passing the bootstrap $ {\cal D}^{(b)} $ as input and using a randomized subset of features selected by the parameter $ {\cal F}^{(b)} $. Eventually, we can also randomize the stop criteria -- governed by the parameters $ k_{max} $ and $ \delta_{min} $ -- or the set of possible decisions evaluated at each dimension $ d $ of the feature space $ {\cal X} $.

````{margin}
```{note}
Typically, we create ensembles containing $ B = 100, 200, 500, \dots $ randomized decision trees and use a third of the features to train each decision tree $ {\cal T}^{(b)} $ such that $ | {\cal F}^{(b)} | = \frac{D}{3} $. 
```
````

### Aggregating

Let $ \lbrace \hat{y}^{(b)} \rbrace $ collect the predicted labels obtained by the ensemble $ {\cal E} $. We can select the final ensemble prediction $ \hat{y} $ by allowing the individual trees $ \lbrace {\cal T}^{(b)} \rbrace $ to vote. Specifically, for an arbitrary feature vector $ {\bf x} $, the Random Forest (RF) classifier selects the most frequent predicted label 
\begin{eqnarray}
 h_{RF}({\bf x}) &=& \argmax_{y \in {\cal Y} } Fr(y; \lbrace \hat{y}^{(b)} \rbrace) \nonumber \\
\hat{y}^{(b)} &=& h_{DT}({\bf x}; {\cal T}^{(b)}), \,\, \forall b \in \lbrace 1, \ldots, B \rbrace, \nonumber
\end{eqnarray}
where the function $ Fr(a; {\cal A}) \triangleq \sum_{a' \in {\cal A}} \left[ a' = a \right] $ counts the frequency of the value $ a $ in a finite set $ {\cal A} $.

```{prf:remark}
Assume that the decision trees in $ {\cal E} $ were designed to perform a regression task instead of a classification task. A decision tree $ {\cal T}^{(b)} \in {\cal E} $ can do that by storing training samples from $ {\cal D}^{(b)} $ at the leaves. In particular, let the training dataset split $$ {\cal D}_{\boldsymbol{\ell}}^{(b)} = \bigcup_{i=1}^{| {\cal D}_{\boldsymbol{\ell}}^{(b)} | } \lbrace \left( {\bf x}_{i}, z_{i} \right) \rbrace $$ collect the data items at the leaf $ \boldsymbol{\ell} $ associated with an input data item $ {\bf x} $. Recap: the binary sequence $ \boldsymbol{\ell} $ indicates which decisions were satisfied ($1$) or not ($0$) by $ {\bf x} $ along the branch from the root to its corresponding leaf and therefore uniquely identifies the leaf itself. We can use e.g. a convex combination $$ h_{DT}({\bf x}; {\cal T}^{(b)}) = \sum_{i=1}^{| {\cal D}_{\boldsymbol{\ell}}^{(b)} |} \alpha_{i}({\bf x}; {\bf x}_{i}) z_{i} $$ to compute the predicted value such that $ \alpha_{i}({\bf x}; {\bf x}_{i}) \propto || {\bf x} - {\bf x}_{i} || $ and $ \sum_{i=1}^{| {\cal D}_{\boldsymbol{\ell}}^{(b)} |} \alpha_{i}({\bf x}; {\bf x}_{i}) = 1 $. In this case, we can combine the predicted values $ \lbrace \hat{z}^{(b)} \rbrace $ obtained by the ensemble $ {\cal E} $ by simply averaging. Thus, for a given feature vector $ {\bf x} $, we can write 
\begin{eqnarray}
 h_{RF}({\bf x}) &=& \frac{1}{B} \sum_{b=1}^{B} \hat{z}^{(b)} \nonumber \\
\hat{z}^{(b)} &=& h_{DT}({\bf x}; {\cal T}^{(b)}), \,\, \forall b \in \lbrace 1, \ldots, B \rbrace. \nonumber
\end{eqnarray}
```

```{prf:remark}
Let $ CE_{test}^{(b)} $ denote the empirical classification error of the DT classifier associated with the $b$-th decision tree $ {\cal T}^{(b)} $ trained based on the boostrap $ {\cal D}^{(b)} $ and evaluated against a testing dataset $ {\cal D}_{test} $. As the testing dataset $ {\cal D}_{test} $ contains realizations of the unknown distribution $ p^{\ast}({\bf x}, y) $, the empirical error $ CE_{test}^{(b)} $ is also a realization of some random variable $ W_{b} \sim p_{b}(w_{b}) $ with variance $ \sigma_{b}^{2} = E_{W_{b} \sim p_{b}} \lbrace (W_{b} - \mu_{b})^{2} \rbrace  $ and mean $ \mu_{b} = E_{W_{b} \sim p_{b}} \lbrace W_{b} \rbrace $. Analogously, we also assume that the empirical classification error $ CE_{test} $ of the ensemble $ {\cal E} $ is a realization of a random variable $ W_{e} \sim p_{e}(w_{e}) $ with variance $ \sigma_{e}^{2} = E_{W_{e} \sim p_{e}} \lbrace (W_{e} - \mu_{e})^{2} \rbrace $ and mean $ \mu_{e} = E_{W_{e} \sim p_{e}} \lbrace W_{e} \rbrace $. Note that, by combining the predicted labels of the ensemble, we expect that the variance $ \sigma_{e}^{2} $ associated with the empirical classification error $ CE_{test} $ of the ensemble will be smaller than any variance $ \lbrace \sigma_{b}^{2} \rbrace $ associated with the classification errors $ \lbrace CE_{test}^{(b)} \rbrace $ of the individual decision trees $ \lbrace {\cal T}^{(b)} \rbrace $. In addition to this, let us assume that in the best case scenario -- for some aggregation procedure -- $ W_{e} $ is given by the average of the random variables $ \lbrace W_{b} \rbrace $, we expect then that -- in this scenario -- the variance $ \sigma_{e}^{2} $ associated with the ensemble $ {\cal E} $ will be the average of the variances $ \lbrace \sigma_{b}^{2} \rbrace $ associated with the individual classifiers. To summarize the discussion, for an arbitrary aggregation procedure, we write $$ \frac{1}{B} \sum_{b=1}^{B} \sigma_{b}^{2} \leq \sigma_{e}^{2} \leq \sigma_{1}^{2}, \sigma_{2}^{2}, \ldots, \sigma_{B}^{2}, $$ i.e. the aggregation procedure reduces the variance associated with the individual classifiers leading to a more robust ensemble classifier in the sense its empirical error $ CE_{test} $ is less sensitive to the employed training and testing datasets: $ {\cal D} $ and $ {\cal D}_{test} $. Finally, it is worth stressing that this discussion aims to motivate the ensemble approach, it does not intend to offer a rigorous proof regarding the ensemble performance.
```